# Patch-based 4K Super-Resolution Training with Advanced Memory Debugging

## 📋 概述

本notebook实现了一个**基于patch的超分辨率训练系统**，专门设计用于在有限GPU显存下训练4K超分辨率模型。

### 🎯 目标任务
- **输入**: 256×256 低分辨率图像
- **输出**: 4096×4096 高分辨率图像
- **放大倍数**: 16× (分两阶段: 8× + 2×)

### 🔑 核心特性

#### 1. **Patch-based训练策略**
- 不训练完整的大图像，而是训练小patch
- LR patch: 64×64 → HR patch: 512×512 (8× scale)
- 训练时显存占用极小 (~1-2GB)
- 推理时使用滑动窗口拼接完整图像

#### 2. **显存监控与调试系统** 🔍
- 实时监控GPU显存使用情况
- 自动测试最大可用batch size
- 详细显示每个训练步骤的显存占用
- OOM错误时给出具体优化建议

#### 3. **自适应优化技术**
- **混合精度训练** (FP16): 减少50%显存占用
- **梯度累积**: 模拟更大batch size而不增加显存
- **轻量级模型**: 最小化参数量和计算量
- **自动batch size调整**: 根据GPU自动选择最优配置

#### 4. **兼容性修复**
- ✅ 修复PyTorch版本兼容性问题
- ✅ 支持旧版和新版autocast API
- ✅ 支持旧版和新版GradScaler API

### 📊 显存占用估算

| 组件 | 显存占用 (batch=1) | 显存占用 (batch=4) |
|------|-------------------|-------------------|
| 模型参数 | ~50 MB | ~50 MB |
| 输入LR (64×64) | 0.05 MB | 0.2 MB |
| 输出HR (512×512) | 3 MB | 12 MB |
| 中间激活 | ~200 MB | ~800 MB |
| 梯度 | ~50 MB | ~50 MB |
| 优化器状态 | ~100 MB | ~100 MB |
| **总计** | **~500 MB** | **~1.2 GB** |

### 🚀 训练流程

```
Step 1: 数据准备
  4096×4096 HR图像 → 随机裁剪512×512 patches
  512×512 HR patch → 下采样得到64×64 LR patch

Step 2: 训练8×模型
  输入: 64×64 LR patch
  输出: 512×512 SR patch
  训练patches: 300张图 × 10 patches/图 = 3000 patches

Step 3: 推理 (256×256 → 2048×2048)
  256×256 → 切成4×4=16个64×64 patches
  → 每个通过8×模型得到512×512
  → 拼接成2048×2048

Step 4: 最后放大 (2048×2048 → 4096×4096)
  使用bicubic插值或轻量2×模型
```

### 📈 预期效果

根据类似任务的经验：
- **训练时间**: ~2-4小时 (50 epochs, RTX 3090)
- **PSNR**: 28-32 dB
- **SSIM**: 0.92-0.96
- **显存占用**: <2GB

### ⚙️ 配置说明

可调整的关键参数：

| 参数 | 默认值 | 作用 | 调整建议 |
|------|-------|------|---------|
| `LR_PATCH_SIZE` | 64 | LR patch大小 | 减小→降低显存 |
| `SCALE_FACTOR` | 8 | 放大倍数 | 固定为8 |
| `BATCH_SIZE` | 自动 | batch大小 | 自动测试 |
| `BASE_CHANNELS` | 24 | 模型通道数 | 减小→降低显存 |
| `GRADIENT_ACCUMULATION` | 2 | 梯度累积步数 | 增加→模拟大batch |

---

## 开始使用

按顺序执行以下cells即可开始训练。系统会自动：
1. 检测GPU显存
2. 测试最优batch size
3. 显示详细的显存使用情况
4. 开始训练并保存最佳模型

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
import random
from tqdm import tqdm
import gc

## 1. GPU显存监控工具

这些工具函数用于实时监控和调试GPU显存使用情况。

### 功能说明：

1. **print_gpu_memory()**: 打印当前GPU显存状态
   - `已分配`: PyTorch当前使用的显存
   - `保留`: PyTorch从CUDA缓存池中保留的显存
   - `峰值`: 训练过程中的最大显存占用

2. **clear_gpu_memory()**: 清理GPU显存
   - 调用Python垃圾回收
   - 清空CUDA缓存
   - 同步CUDA操作

3. **get_tensor_memory()**: 计算单个tensor的显存占用
   - 用于分析哪些tensor占用显存最多

In [ ]:
def print_gpu_memory(tag=""):
    """打印当前GPU显存使用情况"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        max_allocated = torch.cuda.max_memory_allocated() / 1024**3
        print(f"[{tag}] GPU显存: 已分配={allocated:.2f}GB, 保留={reserved:.2f}GB, 峰值={max_allocated:.2f}GB")
        return allocated, reserved, max_allocated
    return 0, 0, 0

def clear_gpu_memory():
    """清理GPU显存"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

def get_tensor_memory(tensor):
    """获取tensor占用的显存(MB)"""
    return tensor.element_size() * tensor.nelement() / 1024**2

## 2. 设备配置与初始状态检查

检测可用的GPU设备，并显示初始显存状态。

### 输出信息：
- GPU型号
- 总显存容量
- 当前显存使用情况

这一步会重置显存统计，确保后续的显存测试准确。

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'使用设备: {device}')

if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f'总显存: {total_memory:.2f} GB')
    
    # 重置显存统计
    torch.cuda.reset_peak_memory_stats()
    clear_gpu_memory()
    print_gpu_memory("初始状态")

## 3. Patch数据集

### Patch-based训练原理

传统的超分辨率训练直接使用完整的大图像，但4K图像(4096×4096)太大，无法放入GPU显存。

**Patch-based方法**通过以下策略解决这个问题：

1. **训练时**: 从大图中随机裁剪小patches
   - 从4096×4096图像中裁剪512×512的HR patch
   - 将HR patch下采样到64×64得到LR patch
   - 每张图可以提取多个patches，增加数据多样性

2. **优势**:
   - 显存占用小（只处理512×512而非4096×4096）
   - 数据增强丰富（每张图产生多个patches）
   - 训练更快（小patch前向/反向传播快）

3. **推理时**: 使用滑动窗口拼接
   - 将大图切成overlapping patches
   - 每个patch独立超分辨率
   - 拼接成完整的大图

### 数据增强

为了提高模型泛化能力，对每个patch应用：
- 随机水平翻转
- 随机垂直翻转  
- 随机旋转90°的倍数

In [ ]:
class PatchSRDataset(Dataset):
    """Patch数据集"""
    def __init__(self, hr_dir, lr_patch_size=64, scale_factor=8, 
                 patches_per_image=10, augment=True):
        self.hr_dir = Path(hr_dir)
        self.lr_patch_size = lr_patch_size
        self.hr_patch_size = lr_patch_size * scale_factor
        self.scale_factor = scale_factor
        self.patches_per_image = patches_per_image
        self.augment = augment
        
        self.hr_images = sorted(list(self.hr_dir.glob('*.png')))
        
        print(f"\n数据集配置:")
        print(f"  图像数量: {len(self.hr_images)}")
        print(f"  LR patch: {lr_patch_size}×{lr_patch_size}")
        print(f"  HR patch: {self.hr_patch_size}×{self.hr_patch_size}")
        print(f"  缩放倍数: {scale_factor}×")
        print(f"  总patches: {len(self.hr_images) * patches_per_image}")
    
    def __len__(self):
        return len(self.hr_images) * self.patches_per_image
    
    def augment_patch(self, lr, hr):
        if random.random() > 0.5:
            lr, hr = np.fliplr(lr), np.fliplr(hr)
        if random.random() > 0.5:
            lr, hr = np.flipud(lr), np.flipud(hr)
        k = random.randint(0, 3)
        if k > 0:
            lr, hr = np.rot90(lr, k), np.rot90(hr, k)
        return lr.copy(), hr.copy()
    
    def __getitem__(self, idx):
        img_idx = idx // self.patches_per_image
        
        hr_img = cv2.imread(str(self.hr_images[img_idx]))
        hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2RGB)
        
        h, w = hr_img.shape[:2]
        max_y, max_x = h - self.hr_patch_size, w - self.hr_patch_size
        
        if max_y <= 0 or max_x <= 0:
            hr_patch = cv2.resize(hr_img, (self.hr_patch_size, self.hr_patch_size))
        else:
            y, x = random.randint(0, max_y), random.randint(0, max_x)
            hr_patch = hr_img[y:y+self.hr_patch_size, x:x+self.hr_patch_size]
        
        lr_patch = cv2.resize(hr_patch, (self.lr_patch_size, self.lr_patch_size), 
                             interpolation=cv2.INTER_CUBIC)
        
        if self.augment:
            lr_patch, hr_patch = self.augment_patch(lr_patch, hr_patch)
        
        lr_tensor = torch.from_numpy(lr_patch.transpose(2, 0, 1)).float() / 255.0
        hr_tensor = torch.from_numpy(hr_patch.transpose(2, 0, 1)).float() / 255.0
        
        return lr_tensor, hr_tensor

## 4. 轻量级U-Net模型架构

### 模型设计原则

为了在有限显存下训练，我们采用以下设计：

#### 1. **减少通道数** 
- 基础通道数设为24（而非常见的64）
- 最深层通道数为96（而非512）
- 参数量减少约75%

#### 2. **移除BatchNorm**
- BatchNorm需要额外显存存储running stats
- 对于小batch size，BN效果不佳
- 使用残差连接保证训练稳定性

#### 3. **浅层编码器**
- 只下采样2次（64→32→16）
- 避免过小的feature map
- 保留更多空间信息

#### 4. **渐进式上采样**
- 从16×16逐步上采样到512×512
- 使用最近邻插值+卷积（比转置卷积省显存）
- 5次2×上采样达到32×放大（16→512）

### 架构流程

```
输入: 64×64×3

编码器:
  64×64×3 → Conv → 64×64×24
  64×64×24 → DownConv → 32×32×48
  32×32×48 → DownConv → 16×16×96

解码器（8×上采样）:
  16×16×96 → Up → 32×32×48
  32×32×48 → Up → 64×64×24
  64×64×24 → Up → 128×128×24
  128×128×24 → Up → 256×256×24
  256×256×24 → Up → 512×512×24
  512×512×24 → Conv1×1 → 512×512×3

输出: 512×512×3
```

### 参数量估算

- 编码器: ~50K parameters
- 解码器: ~200K parameters
- **总计: ~250K parameters** (相比原版UNet减少95%)

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(ch, ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch, ch, 3, padding=1)
        )
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        return self.relu(x + self.conv(x))


class TinyUNet8x(nn.Module):
    """超轻量8×SR模型 (64×64 → 512×512)
    
    设计：最小化显存占用
    """
    def __init__(self, base_ch=24):  # 减少通道数到24
        super().__init__()
        
        # 编码器 (64 → 32 → 16)
        self.inc = nn.Sequential(
            nn.Conv2d(3, base_ch, 3, padding=1),
            nn.ReLU(inplace=True)
        )
        
        self.down1 = nn.Sequential(
            nn.Conv2d(base_ch, base_ch*2, 3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            ResBlock(base_ch*2)
        )
        
        self.down2 = nn.Sequential(
            nn.Conv2d(base_ch*2, base_ch*4, 3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            ResBlock(base_ch*4)
        )
        
        # 上采样到8× (16 → 32 → 64 → 128 → 256 → 512)
        self.up_blocks = nn.ModuleList([
            self._make_up(base_ch*4, base_ch*2),  # 16→32
            self._make_up(base_ch*2, base_ch),    # 32→64
            self._make_up(base_ch, base_ch),      # 64→128
            self._make_up(base_ch, base_ch),      # 128→256
            self._make_up(base_ch, base_ch),      # 256→512
        ])
        
        self.outc = nn.Conv2d(base_ch, 3, 1)
    
    def _make_up(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        x = self.inc(x)
        x = self.down1(x)
        x = self.down2(x)
        
        for up in self.up_blocks:
            x = up(x)
        
        return self.outc(x)

## 5. 训练配置与自适应优化

### 核心配置参数

#### Patch大小
- **LR_PATCH_SIZE**: 64×64 - 低分辨率patch大小
- **HR_PATCH_SIZE**: 512×512 - 高分辨率patch大小  
- **SCALE_FACTOR**: 8× - 本模型的放大倍数

#### 训练超参数
- **BATCH_SIZE**: 自动检测 - 根据GPU显存自动选择
- **GRADIENT_ACCUMULATION**: 2 - 梯度累积步数
  - 有效batch = BATCH_SIZE × GRADIENT_ACCUMULATION
  - 例如: 2 × 2 = 4 (模拟batch=4的效果)
- **LEARNING_RATE**: 1e-4 - 学习率
- **NUM_EPOCHS**: 50 - 训练轮数

#### 模型配置
- **BASE_CHANNELS**: 24 - 基础通道数（越小显存越少）
- **PATCHES_PER_IMAGE**: 10 - 每张图提取的patch数量

### 优化技术详解

#### 1. 梯度累积 (Gradient Accumulation)

**问题**: GPU显存有限，batch size只能设为1或2，训练不稳定

**解决**: 累积多个mini-batch的梯度再更新

```python
for i, (input, target) in enumerate(loader):
    loss = model(input, target) / accumulation_steps
    loss.backward()  # 累积梯度
    
    if (i + 1) % accumulation_steps == 0:
        optimizer.step()  # 更新权重
        optimizer.zero_grad()  # 清空梯度
```

**效果**: batch=2, accumulation=2 ≈ batch=4的训练效果

#### 2. 混合精度训练 (Mixed Precision)

**原理**: 
- FP32: 32位浮点数（高精度，高显存）
- FP16: 16位浮点数（低精度，低显存）
- 混合精度: 大部分操作用FP16，关键操作用FP32

**显存节省**: 约50%

**精度损失**: 几乎没有（<0.1%）

In [ ]:
# 基础配置
LR_PATCH_SIZE = 64
SCALE_FACTOR = 8
HR_PATCH_SIZE = LR_PATCH_SIZE * SCALE_FACTOR

# 训练配置（会自动调整）
INITIAL_BATCH_SIZE = 4  # 从小的batch size开始
GRADIENT_ACCUMULATION = 2  # 梯度累积步数
LEARNING_RATE = 1e-4
NUM_EPOCHS = 50
PATCHES_PER_IMAGE = 10

BASE_CHANNELS = 24  # 减少基础通道数
USE_MIXED_PRECISION = True
TRAIN_SPLIT = 0.9

HR_DIR = './dataset_4k/high_resolution'
CHECKPOINT_DIR = Path('./checkpoints_debug')
CHECKPOINT_DIR.mkdir(exist_ok=True)

print("\n=== 训练配置 ===")
print(f"Patch: {LR_PATCH_SIZE}×{LR_PATCH_SIZE} → {HR_PATCH_SIZE}×{HR_PATCH_SIZE} ({SCALE_FACTOR}×)")
print(f"初始Batch size: {INITIAL_BATCH_SIZE}")
print(f"梯度累积: {GRADIENT_ACCUMULATION} steps")
print(f"有效Batch size: {INITIAL_BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"基础通道数: {BASE_CHANNELS}")
print(f"混合精度: {USE_MIXED_PRECISION}")

## 6. 显存测试与Batch Size自动检测

### 测试目的

在开始训练前，自动测试不同batch size下的显存占用，找出最大可用batch size。

### 测试流程

```
For batch_size in [1, 2, 4, 8]:
    1. 创建随机输入数据
    2. 前向传播 → 记录显存
    3. 计算损失
    4. 反向传播 → 记录显存
    5. 如果OOM，停止测试
```

### 输出信息

对于每个batch size，显示：
- 输入tensor显存占用
- 输出tensor显存占用  
- 前向传播后的总显存
- 反向传播后的峰值显存
- 是否可用

### 自动选择策略

- 选择能成功运行的最大batch size
- 如果最大batch size较小（1-2），建议使用梯度累积

In [ ]:
print("\n=== 显存测试 ===")
print("\n创建模型...")
test_model = TinyUNet8x(base_ch=BASE_CHANNELS).to(device)
total_params = sum(p.numel() for p in test_model.parameters())
print(f"模型参数: {total_params:,}")
print_gpu_memory("模型加载后")

print("\n测试前向传播...")
test_batch_sizes = [1, 2, 4, 8]
max_working_batch = 1

for bs in test_batch_sizes:
    try:
        clear_gpu_memory()
        test_lr = torch.randn(bs, 3, LR_PATCH_SIZE, LR_PATCH_SIZE).to(device)
        test_hr = torch.randn(bs, 3, HR_PATCH_SIZE, HR_PATCH_SIZE).to(device)
        
        print(f"\n测试 batch_size={bs}:")
        print(f"  输入显存: {get_tensor_memory(test_lr):.1f}MB")
        print(f"  目标显存: {get_tensor_memory(test_hr):.1f}MB")
        
        # 测试前向传播
        with autocast(enabled=USE_MIXED_PRECISION):
            out = test_model(test_lr)
            loss = nn.L1Loss()(out, test_hr)
        
        alloc, _, peak = print_gpu_memory(f"前向传播 bs={bs}")
        
        # 测试反向传播
        loss.backward()
        alloc, _, peak = print_gpu_memory(f"反向传播 bs={bs}")
        
        max_working_batch = bs
        print(f"  ✓ batch_size={bs} 可用 (峰值显存: {peak:.2f}GB)")
        
        del test_lr, test_hr, out, loss
        
    except RuntimeError as e:
        if "out of memory" in str(e):
            print(f"  ✗ batch_size={bs} OOM")
            break
        else:
            raise e

clear_gpu_memory()
del test_model

print(f"\n推荐batch size: {max_working_batch}")
BATCH_SIZE = max_working_batch
print(f"使用batch size: {BATCH_SIZE}")

## 7. 数据加载与训练/验证集划分

### 数据加载配置

使用PyTorch的DataLoader加载patch数据集，关键配置：

#### DataLoader参数
- **batch_size**: 使用前面自动检测的最大batch size
- **shuffle**: 训练集shuffle=True，打乱顺序增强泛化
- **num_workers**: 设为0（单进程加载）
  - 多进程加载可能增加显存开销
  - 对于小patch，单进程已足够快
- **pin_memory**: True，加速GPU数据传输

#### 训练/验证集划分
- **训练集**: 90%的patches (用于训练模型)
- **验证集**: 10%的patches (用于监控过拟合)

### 数据量计算

假设有300张4K图像，每张提取10个patches：
- 总patches: 300 × 10 = 3000
- 训练patches: 3000 × 0.9 = 2700
- 验证patches: 3000 × 0.1 = 300

如果batch_size=2：
- 训练batches: 2700 / 2 = 1350
- 验证batches: 300 / 2 = 150

In [ ]:
dataset = PatchSRDataset(
    hr_dir=HR_DIR,
    lr_patch_size=LR_PATCH_SIZE,
    scale_factor=SCALE_FACTOR,
    patches_per_image=PATCHES_PER_IMAGE,
    augment=True
)

train_size = int(TRAIN_SPLIT * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(
    train_set, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=0, pin_memory=True  # num_workers=0更稳定
)
val_loader = DataLoader(
    val_set, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=0, pin_memory=True
)

print(f"\n训练集: {train_size} patches, {len(train_loader)} batches")
print(f"验证集: {val_size} patches, {len(val_loader)} batches")

## 8. 模型与优化器初始化

### 训练组件

#### 1. 模型 (TinyUNet8x)
- 加载到GPU设备
- 基础通道数: 24
- 总参数量: ~250K

#### 2. 损失函数 (L1Loss)
- 也称为MAE (Mean Absolute Error)
- 比MSE更关注大误差，适合图像超分辨率
- 公式: L = |预测 - 真实| 的平均值

#### 3. 优化器 (Adam)
- 学习率: 1e-4 (0.0001)
- Adam自适应调整每个参数的学习率
- 适合处理稀疏梯度和噪声数据

#### 4. 学习率调度器 (CosineAnnealingLR)
- 余弦退火策略
- 学习率从初始值逐渐降低到接近0
- 前期学习快，后期微调
- 公式: lr = lr_min + (lr_max - lr_min) × (1 + cos(π × epoch / T_max)) / 2

#### 5. 梯度缩放器 (GradScaler)
- 仅在混合精度训练时使用
- 防止FP16下梯度下溢
- 自动缩放loss以保持梯度数值稳定性

### 初始化后显存

模型参数占用约50MB显存。

In [ ]:
clear_gpu_memory()
print("\n初始化训练组件...")

model = TinyUNet8x(base_ch=BASE_CHANNELS).to(device)
print_gpu_memory("模型")

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

# 兼容新旧PyTorch版本的GradScaler
if USE_MIXED_PRECISION:
    try:
        # 新版API (PyTorch >= 2.0)
        scaler = torch.amp.GradScaler('cuda')
    except AttributeError:
        # 旧版API (PyTorch < 2.0)
        scaler = GradScaler()
else:
    scaler = None

print(f"模型参数: {sum(p.numel() for p in model.parameters()):,}")

## 9. 训练与验证函数

### train_epoch() - 训练一个epoch

#### 参数说明
- **grad_accum_steps**: 梯度累积步数
- **verbose_memory**: 是否详细打印显存使用

#### 梯度累积实现

```python
optimizer.zero_grad()  # 初始化梯度为0

for i, (lr, hr) in enumerate(loader):
    loss = criterion(out, hr) / grad_accum_steps  # 除以累积步数
    loss.backward()  # 累积梯度（不立即更新权重）

    # 每grad_accum_steps个batch才更新一次
    if (i + 1) % grad_accum_steps == 0:
        optimizer.step()  # 应用累积的梯度
        optimizer.zero_grad()  # 清空梯度
```

**效果**:
- batch_size=2, grad_accum=2 → 等效于batch_size=4
- 显存占用仍然只是batch_size=2的量

#### 混合精度训练流程

```python
with autocast(enabled=True):  # 自动转换为FP16
    out = model(lr)
    loss = criterion(out, hr)

scaler.scale(loss).backward()  # 缩放loss防止梯度下溢
scaler.step(optimizer)  # 应用缩放后的梯度
scaler.update()  # 更新scaler的缩放因子
```

#### 进度显示
- tqdm进度条显示训练进度
- 实时显示当前loss和GPU显存占用
- 第一个epoch显示详细显存信息（每100个batch）

### validate() - 验证函数

- 使用torch.no_grad()禁用梯度计算
- 节省显存和计算时间
- 返回验证集平均loss

In [ ]:
def train_epoch(model, loader, criterion, optimizer, scaler, device, 
                use_amp, grad_accum_steps=1, verbose_memory=False):
    """
    训练一个epoch，带梯度累积和显存监控
    """
    model.train()
    running_loss = 0.0
    optimizer.zero_grad()
    
    pbar = tqdm(loader, desc='训练')
    for i, (lr, hr) in enumerate(pbar):
        lr, hr = lr.to(device), hr.to(device)
        
        # 前向传播
        if use_amp and scaler:
            with autocast(enabled=True):  # 修复：不使用device_type参数
                out = model(lr)
                loss = criterion(out, hr) / grad_accum_steps
            
            scaler.scale(loss).backward()
            
            # 梯度累积
            if (i + 1) % grad_accum_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
        else:
            out = model(lr)
            loss = criterion(out, hr) / grad_accum_steps
            loss.backward()
            
            if (i + 1) % grad_accum_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
        
        running_loss += loss.item() * grad_accum_steps
        
        # 显示显存（每100个batch）
        if verbose_memory and i % 100 == 0:
            alloc, _, peak = print_gpu_memory(f"Batch {i}")
        
        pbar.set_postfix({
            'loss': f'{loss.item() * grad_accum_steps:.4f}',
            'gpu': f'{torch.cuda.memory_allocated()/1024**3:.1f}GB'
        })
    
    return running_loss / len(loader)


def validate(model, loader, criterion, device, use_amp):
    model.eval()
    running_loss = 0.0
    
    with torch.no_grad():
        for lr, hr in tqdm(loader, desc='验证'):
            lr, hr = lr.to(device), hr.to(device)
            
            if use_amp:
                with autocast(enabled=True):  # 修复
                    out = model(lr)
                    loss = criterion(out, hr)
            else:
                out = model(lr)
                loss = criterion(out, hr)
            
            running_loss += loss.item()
    
    return running_loss / len(loader)

## 10. 主训练循环

### 训练流程

对于每个epoch：

1. **训练阶段**
   - 调用train_epoch()在训练集上训练
   - 第一个epoch显示详细显存信息
   - 返回平均训练损失

2. **验证阶段**
   - 调用validate()在验证集上评估
   - 返回平均验证损失

3. **学习率调整**
   - scheduler.step()应用余弦退火

4. **模型保存策略**
   - **最佳模型**: 验证损失最低时保存为`best_model.pth`
     - 包含模型权重、优化器状态、配置信息
   - **定期检查点**: 每10个epoch保存为`epoch_N.pth`
     - 便于恢复训练或分析不同阶段的模型

5. **显存管理**
   - 每5个epoch清理一次GPU缓存
   - 每个epoch结束打印显存统计

### OOM错误处理

如果训练中出现OOM错误，自动显示：
- 当前显存使用情况
- 优化建议：
  1. 减小batch size
  2. 增加梯度累积步数
  3. 减小模型通道数
  4. 减小patch size

### 输出信息

每个epoch显示：
- 训练损失和验证损失
- 当前学习率
- 显存使用情况
- 模型保存状态

In [ ]:
history = {'train_loss': [], 'val_loss': []}
best_val_loss = float('inf')

print("\n" + "="*60)
print("开始训练")
print("="*60)

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    print("-" * 50)
    
    # 第一个epoch显示详细显存
    verbose = (epoch == 0)
    
    try:
        train_loss = train_epoch(
            model, train_loader, criterion, optimizer, scaler, device,
            USE_MIXED_PRECISION, GRADIENT_ACCUMULATION, verbose_memory=verbose
        )
        
        val_loss = validate(model, val_loader, criterion, device, USE_MIXED_PRECISION)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        scheduler.step()
        
        print(f"\n训练损失: {train_loss:.6f}")
        print(f"验证损失: {val_loss:.6f}")
        print(f"学习率: {scheduler.get_last_lr()[0]:.6f}")
        print_gpu_memory(f"Epoch {epoch+1} 结束")
        
        # 保存最佳模型
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'config': {
                    'lr_patch': LR_PATCH_SIZE,
                    'hr_patch': HR_PATCH_SIZE,
                    'scale': SCALE_FACTOR,
                    'base_ch': BASE_CHANNELS
                }
            }, CHECKPOINT_DIR / 'best_model.pth')
            print(f"✓ 保存最佳模型")
        
        # 定期保存
        if (epoch + 1) % 10 == 0:
            torch.save(model.state_dict(), CHECKPOINT_DIR / f'epoch_{epoch+1}.pth')
            print(f"✓ 保存检查点")
        
        # 清理显存
        if (epoch + 1) % 5 == 0:
            clear_gpu_memory()
    
    except RuntimeError as e:
        if "out of memory" in str(e):
            print(f"\n!!! OOM错误 !!!")
            print_gpu_memory("OOM时")
            print("\n建议:")
            print(f"  1. 减小batch size (当前: {BATCH_SIZE})")
            print(f"  2. 增加梯度累积 (当前: {GRADIENT_ACCUMULATION})")
            print(f"  3. 减小BASE_CHANNELS (当前: {BASE_CHANNELS})")
            print(f"  4. 减小patch size (当前: {LR_PATCH_SIZE}→{HR_PATCH_SIZE})")
            raise e
        else:
            raise e

print(f"\n训练完成！最佳验证损失: {best_val_loss:.6f}")

## 11. 训练曲线可视化

### 损失曲线图

绘制训练损失和验证损失随epoch的变化：

- **蓝色线 (Training)**: 训练集损失
  - 应该持续下降
  - 如果不下降，学习率可能太小或模型容量不足

- **橙色线 (Validation)**: 验证集损失
  - 用于判断是否过拟合
  - 如果验证损失上升而训练损失下降 → 过拟合

### 理想曲线特征

✅ **健康的训练**:
- 训练和验证损失都持续下降
- 验证损失略高于训练损失
- 两条曲线走势相似

⚠️ **过拟合警告**:
- 训练损失很低，验证损失很高
- 验证损失开始上升

⚠️ **欠拟合警告**:
- 两个损失都很高且不下降
- 需要增加模型容量或训练更长时间

### 保存

曲线图自动保存到`checkpoints_debug/curve.png`

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history['train_loss'], label='Training', marker='o')
plt.plot(history['val_loss'], label='Validation', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title(f'{SCALE_FACTOR}× SR Training ({LR_PATCH_SIZE}→{HR_PATCH_SIZE})')
plt.legend()
plt.grid(True)
plt.savefig(CHECKPOINT_DIR / 'curve.png', dpi=150)
plt.show()

## 12. 训练总结报告

### 总结内容

生成完整的训练总结，包括：

#### 配置信息
- Patch大小和缩放倍数
- Batch size和梯度累积配置
- 模型通道数和参数量

#### 显存统计
- 训练过程中的峰值显存占用
- 用于评估是否可以进一步增加batch size或模型大小

#### 训练结果
- 最佳验证损失
- 总训练轮数
- 模型保存路径

### 文件输出

总结报告会：
1. 打印到控制台
2. 保存为文本文件: `checkpoints_debug/summary.txt`

### 后续步骤

训练完成后：

1. **加载最佳模型**
```python
checkpoint = torch.load('checkpoints_debug/best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
```

2. **推理测试**
- 使用滑动窗口将256×256图像超分辨率到2048×2048
- 再用bicubic插值放大到4096×4096

3. **质量评估**
- 计算PSNR, SSIM指标
- 与bicubic/其他方法对比

In [ ]:
_, _, peak_memory = print_gpu_memory("训练结束")

summary = f"""
训练总结
{'='*60}

配置:
  Patch: {LR_PATCH_SIZE}×{LR_PATCH_SIZE} → {HR_PATCH_SIZE}×{HR_PATCH_SIZE} ({SCALE_FACTOR}×)
  Batch size: {BATCH_SIZE}
  梯度累积: {GRADIENT_ACCUMULATION}
  有效batch: {BATCH_SIZE * GRADIENT_ACCUMULATION}
  基础通道: {BASE_CHANNELS}
  模型参数: {sum(p.numel() for p in model.parameters()):,}

显存使用:
  峰值显存: {peak_memory:.2f} GB

结果:
  最佳验证损失: {best_val_loss:.6f}
  总训练轮数: {len(history['train_loss'])}

模型保存: {CHECKPOINT_DIR / 'best_model.pth'}
"""

print(summary)

with open(CHECKPOINT_DIR / 'summary.txt', 'w') as f:
    f.write(summary)